In [0]:
%sql
--use f1_processed

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# for delta 
spark.sql(f""" CREATE Table IF NOT EXISTS f1_presentation.calculated_race_results
               (
                 race_year INT,
                 team_name STRING,
                 driver_id Int,
                 driver_name string,
                 race_id int,
                 position int,
                 points int,
                 calculated_points int,
                 created_date TIMESTAMP,
                 updated_date timestamp
               )
               Using DELTA
               """)

DataFrame[]

In [0]:
%sql
-- create table f1_presentation.calculated_race_results
-- using parquet
-- as
-- select races.race_year, 
--        constructors.name as team_name,
--        drivers.name as driver_name,
--        results.position, 
--        results.points,
--        11-results.position as calculated_points
-- from results
-- join f1_processed.drivers On (results.driver_id = drivers.driver_id)
-- join f1_processed.constructors On ( results.constructor_id = constructors.constructor_id)
-- join f1_processed.races On (results.race_id=races.race_id)
-- where results.position<=10       


In [0]:
#for delta 
spark.sql(f""" 
   Create or replace temp view race_results_updated 
   As        
   select races.race_year as race_year,
        races.race_id as race_id, 
        constructors.name as team_name,
        drivers.driver_id as driver_id,
        drivers.name as driver_name,
        results.position, 
        results.points,
        11-results.position as calculated_points
  from f1_processed.results
  join f1_processed.drivers On (results.driver_id = drivers.driver_id)
  join f1_processed.constructors On ( results.constructor_id = constructors.constructor_id)
  join f1_processed.races On (results.race_id=races.race_id)
  where results.position<=10  AND results.file_date = '{v_file_date}'  """)

DataFrame[]

In [0]:
%sql
Select * from race_results_updated

race_year,race_id,team_name,driver_id,driver_name,position,points,calculated_points
2021,1053,Mercedes,1,Lewis Hamilton,2,19.0,9
2021,1053,Alpine F1 Team,4,Fernando Alonso,10,1.0,1
2021,1053,AlphaTauri,842,Pierre Gasly,7,6.0,4
2021,1053,McLaren,817,Daniel Ricciardo,6,8.0,5
2021,1053,Red Bull,830,Max Verstappen,1,25.0,10
2021,1053,Ferrari,832,Carlos Sainz,5,10.0,6
2021,1053,Alpine F1 Team,839,Esteban Ocon,9,2.0,2
2021,1053,Aston Martin,840,Lance Stroll,8,4.0,3
2021,1053,Ferrari,844,Charles Leclerc,4,12.0,7
2021,1053,McLaren,846,Lando Norris,3,15.0,8


In [0]:
%sql
Select * from f1_presentation.calculated_race_results

In [0]:
#Handling Incremental Using Sql merge rather than pyspark 
spark.sql(f"""
merge into f1_presentation.calculated_race_results tgt 
using race_results_updated upd
On (tgt.driver_id =upd.driver_id AND tgt.race_id=upd.race_id)
when Matched then 
  update set tgt.position=upd.position,
              tgt.points=upd.points,
              tgt.calculated_points=upd.calculated_points,
              tgt.updated_date=current_timestamp
when not matched then 
  Insert (race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points,created_date)
  values (race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points,current_timestamp)    """ )         

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
Select count(1) from race_results_updated

count(1)
10


In [0]:
%sql
Select count(1) from f1_presentation.calculated_race_results

count(1)
10055
